# House Price Prediction with MLFLOW

In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()
print(housing)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [3]:
df = pd.DataFrame(housing.data,columns=housing.feature_names)
df["Price"] = housing.target

In [4]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [5]:
X = df.drop(columns=["Price"])
y = df["Price"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [6]:
from urllib.parse import urlparse


In [7]:
signature = infer_signature(X_train,y_train)
signature

inputs: 
  ['MedInc': double (required), 'HouseAge': double (required), 'AveRooms': double (required), 'AveBedrms': double (required), 'Population': double (required), 'AveOccup': double (required), 'Latitude': double (required), 'Longitude': double (required)]
outputs: 
  ['Price': double (required)]
params: 
  None

In [8]:
## Hyperparameter Tuning
def hyperparameter_tuning(model,X_train,y_train,param_grid):
    model =  model
    grid_search = GridSearchCV(estimator=model,param_grid=param_grid,cv=3,verbose=2,scoring="neg_mean_squared_error")
    grid_search.fit(X_train,y_train)
    return grid_search

In [9]:
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["auto", "sqrt", "log2"],
    "bootstrap": [True, False],
    "random_state": [42]
}

lr_param_grid = {
    "fit_intercept": [True, False],
    "copy_X": [True, False],
    "n_jobs": [None, -1],
    "positive": [False, True]
}
rf = RandomForestRegressor()
lr = LinearRegression()

In [10]:
###  Loop through models and log each run
import mlflow.sklearn

# Tracking uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
# It extracts the URI scheme to determine if the MLflow tracking store is local (file) or remote (e.g., http), so the code knows whether model registration is supported.
tracking_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme
for model, param_grid in [(rf, rf_param_grid), (lr, lr_param_grid)]:
    with mlflow.start_run():
        grid_search = hyperparameter_tuning(model, X_train, y_train, param_grid)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        #  Log all best parameters
        mlflow.log_params(grid_search.best_params_)
        
        #  Add tags for clarity
        mlflow.set_tag("model", model.__class__.__name__)

        # Log metric
        mlflow.log_metric("mse", mse)
        
        #  Log model itself
        mlflow.sklearn.log_model(best_model, artifact_path="model")

        
        
        if tracking_uri_type_store != "file":
            mlflow.sklearn.log_model(best_model,"model",registered_model_name=f"Best_{model.__class__.__name__}_model")
        else:
            mlflow.sklearn.log_model(best_model,"model",signature=signature)
        print(f"Best Hyperparameters: {grid_search.best_params_}")
        print(f"Mean Squared Error: {mse}")

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, random_state=42; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, random_state=42; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, random_state=42; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, random_state=42; total time=   0.0s
[CV] EN

d:\MLOPS\venv\mlflow\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
144 fits failed out of a total of 432.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
144 fits failed with the following error:
Traceback (most recent call last):
  File "d:\MLOPS\venv\mlflow\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\MLOPS\venv\mlflow\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "d:\MLOPS\venv\mlflow\lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "d:\MLOPS\venv\mlflow\lib\site-packages\sklearn\utils\_pa

Best Hyperparameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200, 'random_state': 42}
Mean Squared Error: 0.23310245105784574
🏃 View run adaptable-whale-260 at: http://127.0.0.1:5000/#/experiments/0/runs/be758fe685b74b17b19dd7e3a0c62708
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END copy_X=True, fit_intercept=True, n_jobs=None, positive=False; total time=   0.0s
[CV] END copy_X=True, fit_intercept=True, n_jobs=None, positive=False; total time=   0.0s
[CV] END copy_X=True, fit_intercept=True, n_jobs=None, positive=False; total time=   0.0s
[CV] END copy_X=True, fit_intercept=True, n_jobs=None, positive=True; total time=   0.0s
[CV] END copy_X=True, fit_intercept=True, n_jobs=None, positive=True; total time=   0.0s
[CV] END copy_X=True, fit_intercept=True, n_jobs=None, positive=True; total time=   0.0s
[CV] END copy_X=T

2025/05/25 13:41:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/25 13:42:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best_LinearRegression_model'.
2025/05/25 13:42:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best_LinearRegression_model, version 1


Best Hyperparameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}
Mean Squared Error: 0.5411287478470685
🏃 View run victorious-hawk-685 at: http://127.0.0.1:5000/#/experiments/0/runs/9568149ff41043648931e367cd1d7e0d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best_LinearRegression_model'.


In [11]:
print("RF Best params:", grid_search.best_params_)
print("RF MSE:", mse)


RF Best params: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}
RF MSE: 0.5411287478470685
